In [60]:
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_array
import matplotlib.pyplot as plt
import pickle
import warnings
import pandas
import time
import pickle
import networkx as nx
# import hypernetx as hnx
from itertools import chain
import pyalex
import json
import os
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
from OpenAlex import *
from OpenAlexHypergraph import *


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
pyalex.config.email = "l.jiaze@maastrichtuniversity.nl"
pyalex.config.max_retries = 0
pyalex.config.retry_backoff_factor = 0.1
pyalex.config.retry_http_codes = [429, 500, 503]

In [34]:
Works()['W2886742146']

{'id': 'https://openalex.org/W2886742146',
 'doi': 'https://doi.org/10.1109/globalsip.2018.8646701',
 'title': 'FLOW SMOOTHING AND DENOISING: GRAPH SIGNAL PROCESSING IN THE EDGE-SPACE',
 'display_name': 'FLOW SMOOTHING AND DENOISING: GRAPH SIGNAL PROCESSING IN THE EDGE-SPACE',
 'publication_year': 2018,
 'publication_date': '2018-11-01',
 'ids': {'openalex': 'https://openalex.org/W2886742146',
  'doi': 'https://doi.org/10.1109/globalsip.2018.8646701',
  'mag': '2886742146'},
 'language': 'en',
 'primary_location': {'is_oa': False,
  'landing_page_url': 'https://doi.org/10.1109/globalsip.2018.8646701',
  'pdf_url': None,
  'source': None,
  'license': None,
  'license_id': None,
  'version': None,
  'is_accepted': False,
  'is_published': False},
 'type': 'preprint',
 'type_crossref': 'proceedings-article',
 'indexed_in': ['arxiv', 'crossref', 'datacite'],
 'open_access': {'is_oa': True,
  'oa_status': 'green',
  'oa_url': 'https://arxiv.org/pdf/1808.02111',
  'any_repository_has_fullte

# Construct Local Collaboration Hypergraph

In [21]:
alex_works_id, alex_works = Alex.works_from_seed(n_step=2)

Step 0, number_works: 1, ...


Traversing works in 0 step...: 100%|█████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.75s/it]


Step 1, number_works: 215, ...


Traversing works in 1 step...: 100%|█████████████████████████████████████████████████| 215/215 [07:56<00:00,  2.22s/it]


In [31]:
print(len(alex_works_id), alex_works_id[0], len(set(alex_works_id)), type(alex_works), type(alex_works[0]))
all(isinstance(w, pyalex.api.Work) for w in alex_works)
# isinstance(alex_works[0], pyalex.api.Work)

43205 W1992338233 43205 <class 'list'> <class 'pyalex.api.Work'>


True

In [103]:
alex = Alex(works=alex_works)

Loading 43205 works from OpenAlex objects...


Loading works...: 100%|███████████████████████████████████████████████████████| 43205/43205 [00:02<00:00, 19442.92it/s]


Number of Author 101383, Number of Paper 43205 
 Average #_papers per author 3.2801554501247745 
 Average #_coauthors per paper 7.69707209813679
Number of topics 2927.
Most cite 1440 works, least cite 0 works.


In [104]:
path = "./data/LocalCollaborationHypergraph_W3086152724_step2.pkl"  # Hierarchical paper of Jiaze
alex.save(path)

## Hyper CD on Papers 

In [94]:
alex_hyper_paper = HyperPaper(alex)
# alex_hyper_paper.max_component()

There are 61251 authors with 1 paper: number of order_1 hyperedge. To be removed!
There are 0 papers with 0 author: number of degree_0 nodes. To be removed!
Hypergraph on papers, each author is a hyperedge. n=43205, e=40132
 Ks & #hyperedge per order=[[    2     3     4     5     6     7     8     9    10    11    12    13
     14    15    16    17    18    19    20    21    22    23    24    25
     26    27    28    29    30    31    32    33    34    35    36    37
     38    39    40    41    42    43    44    45    46    47    48    49
     50    51    52    53    54    55    56    57    58    59    60    61
     62    63    64    65    66    67    68    69    70    71    72    74
     75    76    77    78    79    80    81    82    83    84    85    86
     87    88    89    90    91    92    93    94    95    96    97    99
    100   101   102   103   104   105   106   107   109   110   112   113
    114   115   116   117   118   119   121   122   123   124   125   128
    129  

In [44]:
hyper_g = hnx.Hypergraph.from_incidence_matrix(alex_hyper_paper.H.toarray())
connected_components = list(hyper_g.s_connected_components(edges=False))

In [47]:
print(len(connected_components))

1


In [48]:
partition_vec, num_groups = HyperCommunityDetect().BetheHessian(alex_hyper_paper, only_assortative=True)
save_path = f"./result/alex_cd/LocalCollaborationHypergraph_W3086152724_step2_BH_q{num_groups}.pkl"
with open(save_path, "wb") as fw:
    pickle.dump(partition_vec, fw)

Constructing HyperBH...: 100%|███████████████████████████████████████████████████████| 244/244 [00:06<00:00, 36.74it/s]


Try first 100 eigenvalue...
Try first 1000 eigenvalue...
number of groups = 655, Kpos=655
EVECs construct: 1551.2000076770782


In [49]:
np.unique(partition_vec, return_counts=True)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [57]:
order = np.argsort(partition_vec)
print_text = "Id AlexId Group Topic\n"
for workid in alex_hyper_paper.nondegree0_row[0][order]:
    print_text += f"{workid} {alex.works_idmap[workid]} {partition_vec[workid]} {alex.works_info[workid]['topic']}\n"

In [58]:
with open(f"./result/alex_cd/LocalCollaborationHypergraph_W3086152724_step2_BH_q{num_groups}.txt", 'w', encoding="utf-8") as f:
    f.write(print_text)

## Visualization

In [75]:
## projection (Too dense)
A = alex_hyper_paper.H.dot(alex_hyper_paper.H.T)
g = nx.from_scipy_sparse_array(A)

In [78]:
proj_graph_linklist = "source;target;value\n"
for l in tqdm(g.edges.data()):
    proj_graph_linklist += f"{l[0]};{l[1]};{l[2]['weight']}\n"

  3%|█▊                                                                   | 268863/9985113 [02:07<1:16:56, 2104.65it/s]


KeyboardInterrupt: 

In [91]:
## Bipartite
# lefttop = csr_array(np.zeros((alex_hyper_paper.n, alex_hyper_paper.n)))
# rightbottom = csr_array(np.zeros((alex_hyper_paper.e, alex_hyper_paper.e)))
# bipartite_A = csr_array(vstack([hstack([lefttop, alex_hyper_paper.H]), hstack([alex_hyper_paper.H.transpose(), rightbottom])]))
bipartite_linklist = "source;target\n"  # from paper to author
rows, cols = alex_hyper_paper.H.nonzero()
for r, c in tqdm(zip(rows, cols)):
    bipartite_linklist += f"{r};{alex_hyper_paper.n+c}\n"

269743it [02:06, 2133.77it/s]


In [92]:
len(alex_hyper_paper.H.nonzero()[0])

269743

In [93]:
with open("./result/visual_graph/LocalCollaborationHypergraph_W3086152724_step2_biGraph.csv", "w") as fw:
    fw.write(bipartite_linklist)

In [117]:
bipartite_meta = "id;type;name;bh_group;alex_url\n"
for i in tqdm(range(alex_hyper_paper.n + alex_hyper_paper.e)):
    line = f"{i};"
    if i < alex_hyper_paper.n:
        title = alex.works_info[i]['title']
        if title is not None:
            title = title.replace(";", " ")
            title = title.replace("\n", " ")
        line += f"paper;{title};{partition_vec[i]};https://openalex.org/works/w{alex.works_idmap[i]}\n"
    else:
        author_id = alex_hyper_paper.nonorder1_column[0][i-alex_hyper_paper.n]
        author_alex_id = alex.authors_idmap[author_id]
        name = alex.authors_info[author_id]["name"]
        name = name.replace(";", " ")
        name = name.replace("\n", " ")
        line += f"author;{name};-1;https://openalex.org/authors/a{author_alex_id}\n"
    bipartite_meta += line

100%|███████████████████████████████████████████████████████████████████████████| 83337/83337 [09:20<00:00, 148.64it/s]


In [118]:
with open("./result/visual_graph/LocalCollaborationHypergraph_W3086152724_step2_biGraph_meta.csv", "w", encoding="utf-8") as fw:
    fw.write(bipartite_meta)

In [119]:
alex.works_info[25229]

{'title': 'Gender differences in collaboration and career progression in physics',
 'citations': [],
 'topic': ('10726', 'Career Development and Diversity'),
 'subfield': ('3311', 'Safety Research'),
 'field': ('33', 'Social Sciences'),
 'domain': ('2', 'Social Sciences'),
 'source': ('4306400194', 'arXiv (Cornell University)')}

In [122]:
alex.incidence_H.T[[25229], :].nonzero()

(array([0, 0, 0, 0]), array([  160,   161, 57906, 57907]))